<a href="https://colab.research.google.com/github/tejatanush/Medical-Charges-Prediction/blob/main/Medical_Charges_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Description
This model is capable of predicting if Medical Chrges prediction based on features like age,BMI, Gender, Smoking, No.of Children..etc . This helps people to get a visualization of how much they will charge lifetime for their health condition. Hospitals and clinics also be helpful by providing information about patient charges quickly.

# Steps to build a model:
1. Import required libraries
2. Import dataset
3. Data Preprocessing
* Find and fill missing values
* Encoding data
* Splitting into training and testing set
* Feature Scaling
4. Selection of model
5. Build a Model
6. Predict Results
7. Evaluate R-Squared score

# 1. Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 2. Import dataset
This dataset has many independent features to predict medical charges required for a patient for lifetime.

Reference:
https://www.kaggle.com/datasets/mirichoi0218/insurance
Lets split our data into two parts x (dependent variables) and y (independent variable).

In [ ]:
dataset=pd.read_csv("https://raw.githubusercontent.com/tejatanush/Medical-Charges-Prediction/refs/heads/main/Health_Price_Prediction.csv")
x=dataset.iloc[:,0:6].values
y=dataset.iloc[:,6].values
print(dataset.head())

# 3. Data Preprocessing

# Find and filling missing values

In [ ]:
missing_values = dataset.isnull().sum()
print(missing_values)

Hence we have no missing values we can skip this step.

# Encoding data
From our data we have categorical features and some label features. So that we should encode categorcal and label encoding in 2 steps.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le=LabelEncoder()
columns_to_encode=[1,4]
for column in columns_to_encode:
    x[:,column]=le.fit_transform(x[:,column])
ct=ColumnTransformer(transformers=[("encoder1",OneHotEncoder(),[3]),("encoder2",OneHotEncoder(),[5])],remainder='passthrough')
x=np.array(ct.fit_transform(x))
x=x.astype('float32')
y=y.astype('float32')

#Splitting into training set and test set
Let's split data so that 80% of data will be training set and remaining 20% will be testing set.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
#Let's check how data splitted
print(x_train)
print(y_train)

We can see that data is not in certain order.... it mean the data is splitted in random.

Feature Scaling
Let's Normalize BMI, Age, Charges.Because these are having unique numerical values and by normalizing them model may understand better patterns between them.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc=MinMaxScaler()
x_train[:,10]=sc.fit_transform((x_train[:,10]).reshape(-1,1)).flatten()
x_test[:,10]=sc.fit_transform((x_test[:,10]).reshape(-1,1)).flatten()
x_train[:,12]=sc.fit_transform((x_train[:,12]).reshape(-1,1)).flatten()
x_test[:,12]=sc.fit_transform((x_test[:,12]).reshape(-1,1)).flatten()
y_train=sc.fit_transform(y_train.reshape(-1,1))
y_test=sc.transform(y_test.reshape(-1,1))
print(x_train[2])
print(y_train[2])

So, the Age,BMI,Charges were normalized into values between 0 and 1.

# 4. Selection of model
Hence the prediction of charges will be many unique values which predicted from many different features. So let's use regression model to predict charges.

#5. Build a model
**Create model:** In regressio problems we make our model from baseline machine learning models. I have already evaluated many models out of them Randonm Forest Regressor model is the best for predicting charges as it showing the good R-Swuared values.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor=RandomForestRegressor(n_estimators=40,random_state=0)


We have constructed the outline of model in a robust manner.

**Fit Model:** Lets fit our model with x_train and y_train data.

In [ ]:
regressor.fit(x_train,y_train)

We have fit our model with x_train and y_train and we can see summary of model.

# 6. Predict Results
Let's Predict the results of x_test and compare them with y_test.

In [ ]:
y_pred=regressor.predict(x_test)
y_pred=y_pred.reshape(-1,1)
y_pred=sc.inverse_transform(y_pred)
y_test=sc.inverse_transform(y_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

Thus the results are very good as they are predicting charges which are closer to original charges. This seems that our model trains good and providing good results.**bold text**

#7. Evaluate model score

In [ ]:
from sklearn.metrics import r2_score
result=r2_score(y_test,y_pred)
print(result)

We are getting a good results that having R-Squared value with nearly 85.5% which mean the model performs very well in predicting charges.